In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
from collections import Counter
from tqdm import tqdm

diretorio = r'Bases/Arquivos_Limpos/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
C_NOMES = ['Estabelecimento', 'Municipio', 'Estado', 'Logradouro', 'Numero']
C_USECOLS = [0, 6, 7, 8, 9]
cartoes = pd.read_excel(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Painel_Leads\BaseCsv\BENVISAVALE.xlsx" , names=C_NOMES, usecols=C_USECOLS)
cartoes.head(5)


,Estabelecimento,Municipio,Estado,Logradouro,Numero
0,GRAN PASTEL GOURMET,BELO HORIZONTE,MG,RUA CARACA,934
1,M130 SERRA,BELO HORIZONTE,MG,R IVAI,130
2,COMBATE DISTRIBUIDORA,BELO HORIZONTE,MG,RUA BAMBUI,56
3,RESTAURANTE GFH 8 EIRELI,BELO HORIZONTE,MG,RUA ORIENTE,2323
4,EPA SUPERMERCADOS,BELO HORIZONTE,MG,R CARACA,600


In [3]:
dtypes = {'CNPJ_BASE': 'object',
 'CNPJ_ORDEM': 'object',
 'CNPJ_DV': 'object'}
df_1 = pd.read_csv(f'{diretorio}{all_files[0]}', 
                names=['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV','NOME_FANTASIA','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'], 
                sep=';',
                encoding='utf-8', 
                usecols=[0,1,2,4, 12, 13, 14, 15], 
                low_memory=False, 
                dtype=dtypes)
print(df_1.columns)
df_1['CNPJ'] = df_1['CNPJ_BASE'].map(str) + df_1['CNPJ_ORDEM'].map(str) + df_1['CNPJ_DV'].map(str)
df_1.drop(['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV'],axis=1, inplace=True)
df_1 = df_1[['CNPJ', 'NOME_FANTASIA', 'LOGRADOURO','NUMERO', 'COMPLEMENTO', 'BAIRRO']]
df_1.head(5)


Index(['CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'NOME_FANTASIA', 'LOGRADOURO',
       'NUMERO', 'COMPLEMENTO', 'BAIRRO'],
      dtype='object')


,CNPJ,NOME_FANTASIA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO
0,39228727000166,AGO RESTOBAR,CLOUTIDES PEREIRA ROCHA,SN,NaN,BAIRRO CEARENSE
1,39228909000137,Indisponível,GETULIO VARGAS,158,NaN,CENTRO
2,39232903000133,PROMESSA DE DEUS,SALAO,681,BLOCO CASA,RUA AUGUSTINHO ALEGRE CHIC
3,39233197000144,PESQUE & PAGUE BOM PESQUEIRO,RODOVIA PR364,SN,NaN,CABECEIRA DO ARROIO GRANDE
4,39237626000151,BAR DO BEM,PARA,440,NaN,PACOVAL


In [4]:

NOMEBEN_OK = {'D_CNPJ':[], 'C_NOME': []}
for c_row in tqdm(cartoes.itertuples(index=False, name='CARTÕES')):
    for d_row in tqdm(df_1.itertuples(index=False, name='CNPJ')):
        if d_row[1] in c_row[0]:
            NOMEBEN_OK['D_CNPJ'].append(d_row[0])
            NOMEBEN_OK['C_NOME'].append(c_row[1])
        else:
            continue

74905it [00:00, 234769.45it/s]
74905it [00:00, 205642.03it/s]
74905it [00:00, 175514.33it/s]
74905it [00:00, 176034.07it/s]
74905it [00:00, 174559.34it/s]
74905it [00:00, 185055.76it/s]
74905it [00:00, 188762.23it/s]
74905it [00:00, 189849.61it/s]
74905it [00:00, 189672.42it/s]
74905it [00:00, 196608.29it/s]
74905it [00:00, 189733.70it/s]
74905it [00:00, 186217.89it/s]
74905it [00:00, 193137.19it/s]
74905it [00:00, 198815.96it/s]
74905it [00:00, 187462.46it/s]
74905it [00:00, 187268.82it/s]
74905it [00:00, 192001.25it/s]
74905it [00:00, 194751.65it/s]
74905it [00:00, 194687.69it/s]
74905it [00:00, 192463.86it/s]
74905it [00:00, 187274.29it/s]
74905it [00:00, 185456.00it/s]
74905it [00:00, 185581.00it/s]
74905it [00:00, 188553.67it/s]
74905it [00:00, 188536.47it/s]
74905it [00:00, 172189.48it/s]
74905it [00:00, 152714.64it/s]
74905it [00:00, 193515.26it/s]
74905it [00:00, 179679.02it/s]
74905it [00:00, 185791.38it/s]
74905it [00:00, 193076.54it/s]
74905it [00:00, 189499.67it/s]
74905it 